In [1]:
import os
import pandas as pd
import numpy as np
from windrose import ProcessTable

In [2]:
# Root directory containing the site directories
root_directory = 'Dataset'

# List of site directories that contain '_FULLSET_' in their names
site_list = [d for d in os.listdir(root_directory) 
             if os.path.isdir(os.path.join(root_directory, d)) and '_FLUXNET_' in d]

# Loop through each site directory
for site in site_list:
    site_path = os.path.join(root_directory, site)

    # List all files in the site directory
    file_list = os.listdir(site_path)

    # Filter files that contain "_FULLSET_" and "_HH_" in their names
    filtered_files = [file for file in file_list if "_FULLSET_" in file and "_HH_" in file]

    # Loop through filtered files and import each as a DataFrame
    for filtered_file in filtered_files:
        file_path = os.path.join(site_path, filtered_file)

        df = pd.read_csv(file_path)

In [3]:
# Drop rows where -9999 occurs in column 'WD' (wind direction)
df_1 = df[df['WD'] != -9999]
# df_1

In [4]:
 # Extract time, speed, and direction
time = df_1['TIMESTAMP_START']  # Assuming YYYYMMDDHHHH format
speed = df_1['WS_F']
direction = df_1['WD']

# Convert time to datetime and extract hour and month
time_series = pd.to_datetime(time, format='%Y%m%d%H%M')
hourz = time_series.dt.hour
monthz = time_series.dt.month

In [5]:
# Create a new DataFrame with the required columns
all_season_df = pd.DataFrame({
    'month': monthz,
    'hour': hourz,
    'speed': speed,
    'direction': direction
})

In [6]:
#defining months for seasons and day and night time hours
all_season = list(range(1, 13))
spring = [3, 4, 5]
summer = [6, 7, 8]
autumn = [9, 10, 11]
winter = [12, 1, 2]
day_time = list(range(6, 18))
night_time = list(range(0, 6)) + list(range(18, 24))
seasons = [all_season, spring, summer, autumn, winter]
day_night = [day_time, night_time]
season_names = ['Full_Year', 'Spring', 'Summer', 'Autumn', 'Winter']
time_names = ['Daytime', 'Nighttime']

In [7]:
# User Defined Parameters
ndirections      = 16

# Calculate 95th percentile of wind speed
percentile95 = np.percentile(speed, 95)

# Wind speed bins
bins1 = [0, 1, 4, 9, 16, 26, 37, 50]
bins2 = np.linspace(0, percentile95, 8)

vwinds = [bins1, bins2] 

In [8]:
for j, vwinds in enumerate(vwinds):
    for c, season in enumerate(seasons):
        season_df = all_season_df[all_season_df['month'].isin(season)]
        
        if c == 0:
            filename = f'{season_names[c]}{j+1}'
            ProcessTable(ndirections, vwinds,season_df, filename)
        else:
            for t, time in enumerate(day_night):
                season_time_df = season_df[season_df['hour'].isin(time)]
                filename = f'{season_names[c]}{time_names[t]}{j+1}'
                ProcessTable(ndirections, vwinds,season_df, filename)